In [80]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/hq/l56ghxv518j9wg6pgqkbbvd80000gn/T/ipykernel_76198/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [81]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [82]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/test/Desktop/ironhack_labs/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
print(data)

data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

                                                   text  label
0     DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                              Will do.      0
2     Nora--Cheryl has emailed dozens of memos about...      0
3     Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                   fyi      0
...                                                 ...    ...
5959  I talked to CDM about doing some strategy sess...      0
5960  Dear Sir/CEO,=20I am Mr.D S Ammer, the purchas...      1
5961                                 Faxing to you now.      0
5962  Goodday,I am Joseph vaye,the son of late Issac...      1
5963  Hello my dearMy name is Prince George Sanoussi...      1

[5964 rows x 2 columns]
(1000, 2)


### Let's divide the training and test set into two partitions

In [94]:
# Define your feature column (e.g., "text") and label column (e.g., "label")
# Replace 'text_column_name' and 'label_column_name' with your actual column names
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/test/Desktop/ironhack_labs/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
print(data)

data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)
X = data['text']   # Features (text)
y = data['label']  # Labels (target)

# Split into train and test sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Confirm the split
print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

data_train = X_train
data_val = X_test
print(data_train) 


                                                   text  label
0     DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                              Will do.      0
2     Nora--Cheryl has emailed dozens of memos about...      0
3     Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                   fyi      0
...                                                 ...    ...
5959  I talked to CDM about doing some strategy sess...      0
5960  Dear Sir/CEO,=20I am Mr.D S Ammer, the purchas...      1
5961                                 Faxing to you now.      0
5962  Goodday,I am Joseph vaye,the son of late Issac...      1
5963  Hello my dearMy name is Prince George Sanoussi...      1

[5964 rows x 2 columns]
(1000, 2)
Training set size: (800,)
Test set size: (200,)
29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on

## Data Preprocessing

In [84]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [85]:
import pandas as pd
import re

# Step 1: Remove inline JS/CSS
def remove_inline_js_css(df):
    def clean_html(text):
        # Remove <script>...</script>
        text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL | re.IGNORECASE)
        # Remove <style>...</style>
        text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL | re.IGNORECASE)
        # Remove inline JavaScript event handlers like onclick="..."
        text = re.sub(r'on\w+="[^"]*"', '', text, flags=re.IGNORECASE)
        # Remove inline styles
        text = re.sub(r'style="[^"]*"', '', text, flags=re.IGNORECASE)
        return text

    return df.apply(clean_html)

data_train = remove_inline_js_css(data_train)
print(data_train)


# Step 2: Remove HTML comments
def remove_html_comments(df):
    def remove_comments(text):
        return re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    return df.apply(remove_comments)

data_train = remove_html_comments(data_train)
print(data_train)


import re

# Step 3: Remove remaining HTML tags
def remove_html_tags(df):
    def strip_tags(text):
        return re.sub(r'<[^>]+>', '', text)
    
    return data_train.apply(strip_tags)


data_train = remove_html_tags(data_train)


print(data_train.head())


29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836        BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...
                             ...                        
106    7653 2612ADAMA IBRAHIM________________________...
270               What does that mean for our schedules?
860    Dear Friend,My Compliment to you,I guess this ...
435    Dear PRESIDENT=2FDIRECTOR=2C My name is Mr=2E ...
102    Let me know if today or tomorrow works for you...
Name: text, Length: 800, dtype: object
29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836        BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...
                             ...                 

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [86]:
# Your code
def clean_text_regex(df):

    def clean(text):
        text = re.sub(r"[^a-zA-Z\s]", " ", text)               # Remove special characters and punctuation
        text = re.sub(r"\b\d+\b", "", text)                    # Remove standalone numbers
        text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)            # Remove single characters
        text = re.sub(r"^[a-zA-Z]\s+", "", text)               # Remove single characters from start
        text = re.sub(r"\s+", " ", text)                       # Replace multiple spaces with single space
        text = re.sub(r"^b\s+", "", text)                      # Remove prefixed 'b' if exists
        return text.lower().strip()                            # Convert to lowercase and strip edges

    return df.apply(clean)

data_train = clean_text_regex(data_train)
print(data_train.head())

29     regards mr nelson smith kindly reply me on my ...
535    have not been able to reach oscar this am we a...
695    huma abedin i checking with pat on the will wo...
557       can have it announced here on monday can today
836    bank of africaagence san pedro bp san pedro co...
Name: text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [87]:
from nltk.corpus import stopwords

def remove_stopwords(df):

    stop_words = set(stopwords.words('english'))

    def filter_stopwords(text):
        tokens = text.split()
        filtered_tokens = [word for word in tokens if word not in stop_words]
        return " ".join(filtered_tokens)

    return df.apply(filter_stopwords)

data_train = remove_stopwords(data_train)

data_train.head(100)


29     regards mr nelson smith kindly reply private e...
535           able reach oscar supposed send pdb receive
695    huma abedin checking pat work jack jake rest a...
557                               announced monday today
836    bank africaagence san pedro bp san pedro cote ...
                             ...                        
468    mrs farah al hashemiaddress chiang rai hospita...
935    dear sir madam mr john coleman sister miss ros...
428    saturday january pm sbwhoeopre fyi foreign nat...
7      abedin huma saturday november pmhfw quint fmsi...
155    xiamen oritent wanlistone company joint ventur...
Name: text, Length: 100, dtype: object

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [92]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

# Download required resources (only needs to be run once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# POS mapping helper function
def get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# Lemmatization function for a single string
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]
    return ' '.join(lemmatized)

# Apply to the DataFrame
data_train = data_train.apply(lemmatize_text)


data_train.head(100)


[nltk_data] Downloading package punkt to /Users/test/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/test/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/test/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


29     regard mr nelson smith kindly reply private em...
535            able reach oscar suppose send pdb receive
695    huma abedin check pat work jack jake rest also...
557                                announce monday today
836    bank africaagence san pedro bp san pedro cote ...
                             ...                        
468    mr farah al hashemiaddress chiang rai hospital...
935    dear sir madam mr john coleman sister miss ris...
428    saturday january pm sbwhoeopre fyi foreign nat...
7      abedin huma saturday november pmhfw quint fmsi...
155    xiamen oritent wanlistone company joint ventur...
Name: text, Length: 100, dtype: object

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [90]:
# Step 1: Initialize the CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

data_train = data_train.astype(str)


# Step 2: Fit and transform the corpus into a BoW representation
X = vectorizer.fit_transform(data_train)
                             
print("Bag of Words:\n", X.toarray())
print("Vocabulary:", vectorizer.get_feature_names_out())

Bag of Words:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]]
Vocabulary: ['102' '106' '270' ... 'zzz' 'zzzahbxntxe' 'zzzj']


## Extra features

In [96]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
#define patterns
#match any of these words or symbols to the text
money_simbol_list = "|".join(["euro","dollar","pound","€","$"]) 
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"]) 

data_train['money_mark'] = data_train.str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train.str.contains(suspicious_words)*1
data_train['text_len'] = data_train.apply(lambda x: len(x)) 

data_val['money_mark'] = data_val.str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val.str.contains(suspicious_words)*1
data_val['text_len'] = data_val.apply(lambda x: len(x)) 

data_train.head()

29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836        BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...
Name: text, dtype: object

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [105]:
# # Your code
# # Step 1: Initialize the TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer()

# # Step 2: Fit and transform the corpus into a TF-IDF representation
# X_tfidf = tfidf_vectorizer.fit_transform(data)

# print(X_tfidf)
# print(X_tfidf.shape)


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()


data_train = data_train.astype(str)
X_tfidf = tfidf_vectorizer.fit_transform(data)

print(X_tfidf)
print(X_tfidf.shape)

                             
# print("Bag of Words:\n", X.toarray())
# print("Vocabulary:", vectorizer.get_feature_names_out())

  (np.int32(0), np.int32(1))	1.0
  (np.int32(1), np.int32(0))	1.0
(2, 2)


## And the Train a Classifier?

In [106]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()


data_train = data_train.astype(str)
X_tfidf = tfidf_vectorizer.fit_transform(data_train)

print(X_tfidf)
print(X_tfidf.shape)

  (np.int32(0), np.int32(17265))	0.19794644495682132
  (np.int32(0), np.int32(14206))	0.17577517319695612
  (np.int32(0), np.int32(14556))	0.4348479070390364
  (np.int32(0), np.int32(18600))	0.36506030424677033
  (np.int32(0), np.int32(12579))	0.2606047392491126
  (np.int32(0), np.int32(17426))	0.2031445982365822
  (np.int32(0), np.int32(13705))	0.128564650907443
  (np.int32(0), np.int32(15253))	0.12327793252226088
  (np.int32(0), np.int32(14311))	0.13105094836221168
  (np.int32(0), np.int32(16514))	0.19521931962375325
  (np.int32(0), np.int32(8601))	0.19311267387964398
  (np.int32(0), np.int32(3495))	0.19575618838274103
  (np.int32(0), np.int32(14557))	0.4826662507702955
  (np.int32(0), np.int32(22844))	0.24802686961780476
  (np.int32(0), np.int32(6642))	0.2031445982365822
  (np.int32(1), np.int32(10634))	0.12327723007352402
  (np.int32(1), np.int32(14753))	0.13923989042630888
  (np.int32(1), np.int32(5410))	0.16214545369483496
  (np.int32(1), np.int32(3230))	0.24369728481835157
  (np

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code